In [102]:
import pandas as pd
import numpy as np
import seaborn as sns

In [103]:
df = pd.read_csv('D1.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1544 entries, 0 to 1543
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   patient_id  1544 non-null   object 
 1   global_num  975 non-null    float64
 2   date        1544 non-null   object 
 3   location    1544 non-null   object 
 4   latitude    1544 non-null   float64
 5   longitude   1544 non-null   float64
dtypes: float64(3), object(3)
memory usage: 72.5+ KB


In [104]:
df.drop(['global_num'],axis=1, inplace=True)
df.drop(['date'],axis=1, inplace=True)
df.drop(['latitude'],axis=1, inplace=True)
df.drop(['longitude'],axis=1, inplace=True)

In [105]:
df

,patient_id,location
0,P1000000501,Chittorgarh_Rajasthan
1,P1000000501,Ratnagiri_Maharashtra
2,P1000000502,Pindwara_Rajasthan
3,P1000000502,Raipur_Chhattisgarh
4,P1000000502,Gokak_Karnataka
...,...,...
1539,P9011000510,Sagar_Karnataka
1540,P9011000510,Jorapokhar_Jharkhand
1541,P9014000503,Rewa_Madhya Pradesh
1542,P9014000504,Dharmavaram_Andhra Pradesh


In [106]:
#group by account
transactions = df.groupby(['patient_id'])['location'].apply(list)
print(transactions.head(10))

patient_id
P1000000501       [Chittorgarh_Rajasthan, Ratnagiri_Maharashtra]
P1000000502    [Pindwara_Rajasthan, Raipur_Chhattisgarh, Goka...
P1000000504                              [Lucknow_Uttar Pradesh]
P1000000505                              [Lucknow_Uttar Pradesh]
P1000000506                                        [Delhi_Delhi]
P1000000507                                        [Delhi_Delhi]
P1000000508                              [Ratnagiri_Maharashtra]
P1000000509                              [Lucknow_Uttar Pradesh]
P1000000510                                 [Mumbai_Maharashtra]
P1000000511             [Ratnagiri_Maharashtra, Sagar_Karnataka]
Name: location, dtype: object


In [107]:
from apyori import apriori

transaction_list = list(transactions)
results = list(apriori(transaction_list, min_support=0.001, min_confidence=0.01))#Q2min_support=0.01, min_confidence=0.01
print(results[:5])

[RelationRecord(items=frozenset({'Alipurduar_West Bengal'}), support=0.026344676180021953, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Alipurduar_West Bengal'}), confidence=0.026344676180021953, lift=1.0)]), RelationRecord(items=frozenset({'Anand_Gujarat'}), support=0.01646542261251372, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Anand_Gujarat'}), confidence=0.01646542261251372, lift=1.0)]), RelationRecord(items=frozenset({'Asansol_West Bengal'}), support=0.018660812294182216, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Asansol_West Bengal'}), confidence=0.018660812294182216, lift=1.0)]), RelationRecord(items=frozenset({'Barh_Bihar'}), support=0.012074643249176729, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'Barh_Bihar'}), confidence=0.012074643249176729, lift=1.0)]), RelationRecord(items=frozenset({'Basirhat_West Bengal'}), support

In [108]:
def convert_apriori_results_to_pandas_df(results):
    rules = []
    
    for rule_set in results:
        for rule in rule_set.ordered_statistics:
            # items_base = left side of rules, items_add = right side
            # support, confidence and lift for respective rules
            rules.append([','.join(rule.items_base), ','.join(rule.items_add),
                         rule_set.support, rule.confidence, rule.lift]) 
    
    # typecast it to pandas df
    return pd.DataFrame(rules, columns=['Left_side', 'Right_side', 'Support', 
                                        'Confidence', 'Lift']) 

result_df = convert_apriori_results_to_pandas_df(results)

print(result_df.head(20))
df['location'].value_counts()

   Left_side                 Right_side   Support  Confidence  Lift
0                Alipurduar_West Bengal  0.026345    0.026345   1.0
1                         Anand_Gujarat  0.016465    0.016465   1.0
2                   Asansol_West Bengal  0.018661    0.018661   1.0
3                            Barh_Bihar  0.012075    0.012075   1.0
4                  Basirhat_West Bengal  0.010977    0.010977   1.0
5                         Batumi_Adjara  0.010977    0.010977   1.0
6                     Belgaum_Karnataka  0.015368    0.015368   1.0
7             Bhimavaram_Andhra Pradesh  0.012075    0.012075   1.0
8                      Chalakudy_Kerala  0.055982    0.055982   1.0
9                 Channapatna_Karnataka  0.055982    0.055982   1.0
10                     Chatrapur_Odisha  0.023052    0.023052   1.0
11                Chittorgarh_Rajasthan  0.010977    0.010977   1.0
12                          Delhi_Delhi  0.019759    0.019759   1.0
13                      Gokak_Karnataka  0.05488

Sardarshahar_Rajasthan      134
Lucknow_Uttar Pradesh        79
Ranebennur_Karnataka         79
Ratnagiri_Maharashtra        57
Kollam_Kerala                54
                           ... 
Banswara_Rajasthan            1
Gudur_Andhra Pradesh          1
Bardoli_Gujarat               1
Ichalkaranji_Maharashtra      1
Karnal_Haryana                1
Name: location, Length: 151, dtype: int64

In [109]:
# sort all acquired rules descending by lift
result_df1 = result_df.loc[result_df['Left_side'] == 'Ranebennur_Karnataka'].sort_values(by='Lift', ascending=False)

print(result_df1.head(5))

                 Left_side                                         Right_side  \
2883  Ranebennur_Karnataka         Jorapokhar_Jharkhand,Ratnagiri_Maharashtra   
4355  Ranebennur_Karnataka  Channapatna_Karnataka,Medinipur_West Bengal,Ra...   
4314  Ranebennur_Karnataka  Ramanagara_Karnataka,Channapatna_Karnataka,Mah...   
4271  Ranebennur_Karnataka  Jorapokhar_Jharkhand,Channapatna_Karnataka,Rat...   
4216  Ranebennur_Karnataka  Jorapokhar_Jharkhand,Channapatna_Karnataka,Mah...   

       Support  Confidence       Lift  
2883  0.002195    0.025316  11.531646  
4355  0.001098    0.012658  11.531646  
4314  0.001098    0.012658  11.531646  
4271  0.001098    0.012658  11.531646  
4216  0.001098    0.012658  11.531646  


In [110]:
result_df = result_df.sort_values(by='Lift', ascending=False)
print(result_df.head(10))

                                              Left_side  \
5308  Chalakudy_Kerala,Bokaro Steel City_Jharkhand,C...   
5129  Basirhat_West Bengal,Delhi_Delhi,Manikchak_Wes...   
5132  Basirhat_West Bengal,Gokak_Karnataka,Rasayani_...   
4703              Suryapet_Telangana,Junnar_Maharashtra   
5134  Manikchak_West Bengal,Delhi_Delhi,Gokak_Karnataka   
5135   Delhi_Delhi,Rasayani_Maharashtra,Gokak_Karnataka   
5136  Delhi_Delhi,Rasayani_Maharashtra,Manikchak_Wes...   
4704                 Suryapet_Telangana,Mehsana_Gujarat   
5137  Gokak_Karnataka,Rasayani_Maharashtra,Manikchak...   
1732                        Bokaro Steel City_Jharkhand   

                                      Right_side   Support  Confidence   Lift  
5308         Yavatmal_Maharashtra,Pali_Rajasthan  0.001098         1.0  911.0  
5129        Gokak_Karnataka,Rasayani_Maharashtra  0.001098         1.0  911.0  
5132           Delhi_Delhi,Manikchak_West Bengal  0.001098         1.0  911.0  
4703              Mehsana_Guja

In [111]:
transactions = df.groupby(['patient_id'])['location'].apply(list)
sequences = transactions.values.tolist()

# show the first 5 sequences
print(sequences[:5])

[['Chittorgarh_Rajasthan', 'Ratnagiri_Maharashtra'], ['Pindwara_Rajasthan', 'Raipur_Chhattisgarh', 'Gokak_Karnataka'], ['Lucknow_Uttar Pradesh'], ['Lucknow_Uttar Pradesh'], ['Delhi_Delhi']]


In [113]:
from collections import defaultdict
import subprocess
import re

''' Uses SPMF to find association rules in supplied transactions '''
def get_association_rules(sequences, min_sup, min_conf):
    # step 1: create required input for SPMF
    
    # prepare a dict to uniquely assign each item in the transactions to an int ID
    item_dict = defaultdict(int)
    output_dict = defaultdict(str)
    item_id = 1
    
    # write your sequences in SPMF format
    with open('seq_rule_input.txt', 'w+') as f:
        for sequence in sequences:
            z = []
            for itemset in sequence:
                # if there are multiple items in one itemset
                if isinstance(itemset, list):
                    for item in itemset:
                        if item not in item_dict:
                            item_dict[item] = item_id
                            item_id += 1

                        z.append(item_dict[item])
                else:
                    if itemset not in item_dict:
                        item_dict[itemset] = item_id
                        output_dict[str(item_id)] = itemset
                        item_id += 1
                    z.append(item_dict[itemset])
                    
                # end of itemset
                z.append(-1)
            
            # end of a sequence
            z.append(-2)
            f.write(' '.join([str(x) for x in z]))
            f.write('\n')
    
    # run SPMF with supplied parameters
    supp_param = '{}%'.format(int(min_sup * 100))
    conf_param = '{}%'.format(int(min_conf * 100))
    subprocess.call(['java', '-jar', 'spmf.jar', 'run', 'RuleGrowth', 'seq_rule_input.txt', 'seq_rule_output.txt', supp_param, conf_param], shell=True)
    
    # read back the output rules
    outputs = open('seq_rule_output.txt', 'r').read().strip().split('\n')
    output_rules = []
    for rule in outputs:
        left, right, sup, conf = re.search(pattern=r'([0-9\,]+) ==> ([0-9\,]+) #SUP: ([0-9]+) #CONF: ([0-9\.]+)', string=rule).groups()
        sup = int(sup) / len(sequences)
        conf = float(conf)
        output_rules.append([[output_dict[x] for x in left.split(',')], [output_dict[x] for x in right.split(',')], sup, conf])
    
    # return pandas DataFrame
    return pd.DataFrame(output_rules, columns = ['Left_rule', 'Right_rule', 'Support', 'Confidence'])

In [115]:
#set min_sup, min_conf as 0.01 and 0.01
get_association_rules(sequences, 0.01, 0.01)

,Left_rule,Right_rule,Support,Confidence
0,[Sardarshahar_Rajasthan],[Gokak_Karnataka],0.026345,0.179104
1,[Sardarshahar_Rajasthan],[Lucknow_Uttar Pradesh],0.018661,0.126866
2,[Sardarshahar_Rajasthan],[Kollam_Kerala],0.012075,0.082090
3,[Channapatna_Karnataka],[Jorapokhar_Jharkhand],0.010977,0.196078
